# Import package

In [1]:
import tensorflow as tf

import numpy as np

import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
# import seaborn as sns
#import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import numpy as np
import PIL
#import requests
from io import BytesIO
from IPython.display import display
%matplotlib inline

#from mpl_toolkits.mplot3d import Axes3D
# from sklearn import datasets
# from sklearn.decomposition import PCA
# from sklearn.utils import shuffle

tf.enable_eager_execution()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation

import os
import scipy

# Import data

In [29]:
# Get the name of all the labels
home_path = '/home/bowen/Documents/Ecomed/'
data_path = '/home/bowen/Documents/Ecomed/dataset'
class_names = os.listdir(data_path)
class_names

['background',
 'needles_I_V_Cannula',
 'needles_safety',
 'needles_black',
 'needles_yellow',
 'garbage_gloves',
 'syringes_large',
 'garbage_alcohol_pad',
 'needles_butterfly',
 'syringes_small',
 'needles_pink',
 'syringes_1ml',
 'garbage_cotton_ball',
 'needles_blood_collection']

In [82]:
# Set up data generators that can read images from our dataset into Keras.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

data_generator = datagen.flow_from_directory(
        data_path,
        target_size=(224, 224),
        batch_size=1000,
        class_mode='binary')

Found 2033 images belonging to 14 classes.


# Convert Model

In [23]:
model_path = os.path.join('/home/bowen/Documents/Ecomed/models', '1550686223')
# model = tf.keras.models.load_model(model_path)

In [24]:
# Converting from keras is super simple, we simply point a tfliteconverter to our model!
converter = tf.contrib.lite.TFLiteConverter.from_saved_model(model_path)

INFO:tensorflow:Restoring parameters from /home/bowen/Documents/Ecomed/models/1550686223/variables/variables
INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default'}
INFO:tensorflow:input tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: input_7
INFO:tensorflow: tensor name: input_7:0, shape: (-1, 224, 224, 3), type: DT_FLOAT
INFO:tensorflow:output tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: Logits
INFO:tensorflow: tensor name: Logits/Softmax:0, shape: (-1, 1000), type: DT_FLOAT
INFO:tensorflow:Restoring parameters from /home/bowen/Documents/Ecomed/models/1550686223/variables/variables
INFO:tensorflow:Froze 262 variables.
INFO:tensorflow:Converted 262 variables to const ops.


In [25]:
tflite_model = converter.convert()

In [26]:
# Specify where to save the new model.
tflite_model_path = os.path.join('/home/bowen/Documents/Ecomed/models', 'model.tflite')
# Write the bytes of the model file to our drive.
open(tflite_model_path, "wb").write(tflite_model)
# Let's just check the size of the model.
print("Model size is %d KBytes." % (os.stat(tflite_model_path).st_size/1000))

Model size is 13973 KBytes.


# Test the lite model

In [64]:
# Create a tflite interpreter, this is the part of tflite that actually runs models.
interpreter = tf.contrib.lite.Interpreter(model_path=tflite_model_path)
# Allocate memory for the all the weight tensors and such.
interpreter.allocate_tensors()
# Get input and output tensor information.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'input_7', 'index': 181, 'shape': array([  1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
[{'name': 'Logits/Softmax', 'index': 9, 'shape': array([   1, 1000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


In [65]:
def run_interpreter(input_data):
    interpreter.reset_all_variables()
    # Set tensor specifies the input to the model.
    interpreter.set_tensor(input_details[0]['index'], input_data)
    # Invoke is used to run the input through the model.
    interpreter.invoke()
    # We have to use get_tensor to load the output.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return np.argmax(output_data)

In [87]:
test_data, test_labels = data_generator.next()

In [88]:
correct = 0
total = 0
for i, sample in enumerate(test_data):
    input_data = sample[tf.newaxis, ...]
    predict = run_interpreter(input_data)
    if predict == int(test_labels[i]):
        correct +=1
    total +=1
print("Total Samples", total)
print("accuracy", correct / total)

Total Samples 1000
accuracy 0.992
